In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
    border-style: groove;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: au                              to;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Module 7 - A panel of experts
</center>
</h1>
<div class=h1_cell>
<p>
This is the last module where we will be treating words as numbers, in essence, vectorizing words. I want to leave this part of the course with a bang so I decided to look at a real problem. We will be back to doing prediction. The problem is that the site donorschoose.org is overwhelmed with applications. You can read the details at https://www.kaggle.com/c/donorschoose-application-screening (yes, this is a Kaggle competition). Briefly, teachers apply for funding to help them further their classroom teaching. The application has a number of boxes for a teacher to fill in. We will concentrate on just three: the first essay they must write, the second essay they must write and the title of their project. Our target column is 'project_is_funded', a binary column with 0 meaning their application was rejected.
<p>
I am going to add a bit of a twist this week. I am going to ask you to use what is called an *ensemble* approach. At its broadest, this ia a machine learning term meaning you will use a number of models/methods to make a prediction. They are the ensemble. They reach a prediction by simple voting. They can be all the same methods (like we will use) or different methods (e.g., mixture of NB and KNN). Ensemble methods have had huge success in machine learning in general so I decided to try it here. Below is the general idea:
<ul>
<li> For essay 1, use NB using only essay 1 words.
<li> For essay 2, use NB using only essay 2 words.
<li> For the title, use NB using only title words.
</ul>
<p>
That is our motley ensemble. Each NB method specializes in one of the columns, They will each reach a prediction then vote.
<p>
Note: you will not really create three separate NB functions. You will simply call your one NB function with different parameters.

</div>

<h2>
What you will need
</h2>
<div class=h1_cell>
<p>
You will need the following functions from past modules:
<ul>
<li> sentence_wrangler and its helpers.
<li> naive_bayes using cosine_similarity
</ul>
<p>
You will also need patience given that we will be using 180K separate teacher-applications.

</div>

In [31]:
import pandas as pd
url = 'https://www.dropbox.com/s/2hdbltrl8bh6kbu/train.csv?raw=1'
donate_table = pd.read_csv(url, encoding='utf-8')

KeyboardInterrupt: 

In [3]:
donate_table.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1


In [4]:
donate_table.describe(include='all')

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
count,182080,182080,182076,182080,182080,182080,182080,182080,182080,182080,182080,6374,6374,182080,182080.000000,182080.000000
unique,182080,104414,5,51,180439,4,51,407,164282,147689,180984,6359,6336,179730,NaN,NaN
top,p009803,fa2f220b537e8653fb48878ebb38044d,Mrs.,CA,2016-09-01 00:00:03,Grades PreK-2,Literacy & Language,Literacy,Flexible Seating,As a teacher in a low-income/high poverty scho...,Students will be using Chromebooks to increase...,"My students need to learn how to rhyme, identi...","As a teacher, it is important to show students...",My students need electronic tablets to do all ...,NaN,NaN
freq,1,74,95405,25695,30,73890,39257,15775,377,46,24,2,3,84,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.237055,0.847682
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.016086,0.359330
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,1.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.000000,1.000000


In [5]:
len(donate_table)

182080

In [6]:
row0_essay1 = donate_table.loc[0, 'project_essay_1']
row0_essay1

u'Most of my kindergarten students come from low-income households and are considered \\"at-risk\\". These kids walk to school alongside their parents and most have never been further than walking distance from their house. For 80% of my students, English is not their first language or the language spoken at home. \\r\\n\\r\\nWhile my kindergarten kids have many obstacles in front of them, they come to school each day excited and ready to learn. Most students started the year out never being in a school setting. At the start of the year many had never been exposed to letters. Each day they soak up more knowledge and try their hardest to succeed. They are highly motivated to learn new things every day. We are halfway through the year and they are starting to take off. They know know all letters, some sight words, numbers to 20, and a majority of their letter sounds because of their hard work and determination. I am excited to see the places we will go from here!'

In [7]:
sentences = row0_essay1.split('.')  # I am using a simple split on period
sentences

[u'Most of my kindergarten students come from low-income households and are considered \\"at-risk\\"',
 u' These kids walk to school alongside their parents and most have never been further than walking distance from their house',
 u' For 80% of my students, English is not their first language or the language spoken at home',
 u' \\r\\n\\r\\nWhile my kindergarten kids have many obstacles in front of them, they come to school each day excited and ready to learn',
 u' Most students started the year out never being in a school setting',
 u' At the start of the year many had never been exposed to letters',
 u' Each day they soak up more knowledge and try their hardest to succeed',
 u' They are highly motivated to learn new things every day',
 u' We are halfway through the year and they are starting to take off',
 u' They know know all letters, some sight words, numbers to 20, and a majority of their letter sounds because of their hard work and determination',
 u' I am excited to see the pl

In [8]:
#sentence_wrangler here

def sentence_wrangler(sentence, swords, legal_chars):
    tokes = word_punct_tokenizer.tokenize(sentence)
    result = []
    removed = []
    check = 0
    for word in tokes:
        if word.lower() not in swords:
            for i in word.lower():
                if i in legal_chars:
                    check = 1
                else: 
                    check = 0
            if check == 1:
                result.append(word.lower())
            else:
                removed.append(word.lower())
        else:
            removed.append(word.lower())
    return (result, removed)

In [9]:
#stuff need to wrangle here
import string
punctuation = string.punctuation

legals = 'abcdefghijklmnopqrstuvwxyz'

from nltk.corpus import stopwords
swords = stopwords.words('english')

from nltk.tokenize import WordPunctTokenizer
word_punct_tokenizer = WordPunctTokenizer()


In [10]:
sentence_wrangler(sentences[0], swords, legals)  # sentences created above

([u'kindergarten',
  u'students',
  u'come',
  u'low',
  u'income',
  u'households',
  u'considered',
  u'risk'],
 [u'most',
  u'of',
  u'my',
  u'from',
  u'-',
  u'and',
  u'are',
  u'\\"',
  u'at',
  u'-',
  u'\\"'])

In [11]:
title0 = donate_table.loc[0, 'project_title']  # no need to break into sentence
title0

u'Super Sight Word Centers'

In [12]:
sentence_wrangler(title0, swords, legals)  # nothing removed

([u'super', u'sight', u'word', u'centers'], [])

<h2>
Generate the bag of words you will need for essay 1, essay 2, and title.
</h2>
<div class=h1_cell>
Remember you need 3 separate bag of words: feed them from the 2 separate essays and the title.

</div>

In [13]:
import time

start = time.time()

bag_of_essay1 = {}
bag_of_essay2 = {}
bag_of_title = {}
count = 0
needed = ["project_title", "project_essay_1", "project_essay_2", "project_is_approved"]
for i in range(len(donate_table)):
    vec = donate_table.loc[i, needed]
    title = sentence_wrangler(vec["project_title"], swords, legals)[0]
    essay_1 = sentence_wrangler(vec["project_essay_1"], swords, legals)[0]
    essay_2 = sentence_wrangler(vec["project_essay_2"], swords, legals)[0]
    for word in title:
        if word not in bag_of_title:
            if vec["project_is_approved"] == 1:
                bag_of_title[word] = [0,1]
            else:
                bag_of_title[word] = [1,0]
        else:
            bag_of_title[word][vec["project_is_approved"]] += 1
    for word in essay_1:
        if word not in bag_of_essay1:
            if vec["project_is_approved"] == 1:
                bag_of_essay1[word] = [0,1]
            else:
                bag_of_essay1[word] = [1,0]
        else:
            bag_of_essay1[word][vec["project_is_approved"]] += 1
    for word in essay_2:
        if word not in bag_of_essay2:
            if vec["project_is_approved"] == 1:
                bag_of_essay2[word] = [0,1]
            else:
                bag_of_essay2[word] = [1,0]
        else:
            bag_of_essay2[word][vec["project_is_approved"]] += 1
        
    if i%4000 == 0: 
        count += 1
        print('%d/46' % count)
        
            
end = time.time()
print(end - start)  # roughly 6 minutes

1/46
2/46
3/46
4/46
5/46
6/46
7/46
8/46
9/46
10/46
11/46
12/46
13/46
14/46
15/46
16/46
17/46
18/46
19/46
20/46
21/46
22/46
23/46
24/46
25/46
26/46
27/46
28/46
29/46
30/46
31/46
32/46
33/46
34/46
35/46
36/46
37/46
38/46
39/46
40/46
41/46
42/46
43/46
44/46
45/46
46/46
651.554775953


In [14]:
len(bag_of_essay1)

41222

In [15]:
bag_of_essay1.items()[:10]

[(u'uwva', [0, 1]),
 (u'tilton', [0, 1]),
 (u'woods', [18, 89]),
 (u'spiders', [0, 5]),
 (u'hanging', [20, 75]),
 (u'localized', [0, 1]),
 (u'canes', [0, 1]),
 (u'sprague', [0, 2]),
 (u'chatter', [10, 62]),
 (u'scold', [0, 1])]

In [16]:
len(bag_of_essay2)

57497

In [17]:
bag_of_essay2.items()[:10]

[(u'fawn', [1, 0]),
 (u'refreshable', [0, 1]),
 (u'preparesee', [0, 1]),
 (u'askew', [0, 1]),
 (u'circuitry', [20, 110]),
 (u'spiders', [3, 30]),
 (u'hanging', [48, 310]),
 (u'woody', [0, 6]),
 (u'websited', [1, 0]),
 (u'localized', [0, 1])]

In [18]:
len(bag_of_title)

21372

In [19]:
bag_of_title.items()[:10]

[(u'fawn', [0, 1]),
 (u'raining', [1, 19]),
 (u'writings', [0, 2]),
 (u'ekk', [0, 1]),
 (u'ncatch', [0, 1]),
 (u'nauthor', [0, 1]),
 (u'yellow', [3, 8]),
 (u'four', [6, 50]),
 (u'rocketing', [0, 4]),
 (u'woods', [4, 8])]

<h2>
Generate the counts you will need for Naive Bayes
</h2>
<div class=h1_cell>
<p>
You need counts and probabilities for applications being approved or not.

</div>

In [20]:
count = 0.0
classc = 0.0
for i in range(len(donate_table)):
    vec = donate_table.loc[i, 'project_is_approved']
    if vec == 1:
        classc += 1
    count += 1
useful_counts = {}
useful_counts['class_count'] = (count-classc, classc)
useful_counts['class_prob'] = ((count-classc)/count, (classc/count))
useful_counts['donate_count'] = count
useful_counts

{'class_count': (27734.0, 154346.0),
 'class_prob': (0.1523176625659051, 0.8476823374340949),
 'donate_count': 182080.0}

In [21]:
#your cosine similarity here
def cosine_similarity(v1,v2):
    AB = 0.0
    A = 0.0
    B = 0.0
    for i in range(len(v1)):
        AB += (v1[i] * v2[i])
        A += (v1[i]**2)
        B += (v2[i]**2)  
    return AB/((A*B)**(0.5))


In [22]:
#your naive bayes here
def naive_bayes(data, bag, counts):
    tracker = []
    for i in range(len(counts['class_prob'])):
        tracker.append(counts['class_prob'][i])
    if isinstance(data, unicode):
        for word in sentence_wrangler(data, swords, legals)[0]:
            if word in bag:
                for i in range(len(counts['class_prob'])):
                    probability = float(bag[word][i])/counts['class_count'][i]
                    tracker[i] *= probability
    else:
        for word in data:
            if word in bag:
                for i in range(len(counts['class_prob'])):
                    probability = float(bag[word][i])/counts['class_count'][i]
                    tracker[i] *= probability
    return tuple(tracker)


<h2>
Test your essay 1 NB
</h2>
<div class=h1_cell>
<p>
Try it for row 0. Print actual value and your predicted value.

</div>

In [23]:
predict = naive_bayes(donate_table.loc[0, 'project_essay_1'], bag_of_essay1, useful_counts)
p = {}
a = {}
if predict[0] > predict[1]:
    p['prediction'] = 0
else:
    p['prediction'] = 1
a['actual'] = donate_table.loc[0, 'project_is_approved']
print predict
print p
print a

(1.5539617092832785e-95, 9.791535781008517e-94)
{'prediction': 1}
{'actual': 1}


<h2>
Test your essay 2 NB
</h2>
<div class=h1_cell>
<p>
Try it for row 0. Print actual value and your predicted value.

</div>

In [24]:
predict = naive_bayes(donate_table.loc[0, 'project_essay_2'], bag_of_essay2, useful_counts)
p = {}
a = {}
if predict[0] > predict[1]:
    p['prediction'] = 0
else:
    p['prediction'] = 1
a['actual'] = donate_table.loc[0, 'project_is_approved']
print predict
print p
print a

(2.1759687231877777e-87, 1.3477242718465147e-82)
{'prediction': 1}
{'actual': 1}


<h2>
Test your title NB
</h2>
<div class=h1_cell>
<p>
Try it for row 0. Print actual value and your predicted value.


</div>

In [66]:
predict = naive_bayes(donate_table.loc[0, 'project_title'], bag_of_title, useful_counts)
p = {}
a = {}
if predict[0] > predict[1]:
    p['prediction'] = 0
else:
    p['prediction'] = 1
a['actual'] = donate_table.loc[0, 'project_is_approved']
print predict
print p
print a

(8.94705022388725e-12, 2.1026684184597748e-11)
{'prediction': 1}
{'actual': 1}


In [67]:
cosine_similarity(bag_of_title['calm'], bag_of_title['word'])

0.9999890362045437

<h2>
Ok, the big show
</h2>
<div class=h1_cell>
<p>
Here is general strategy. Go through every row of donate_table. For each row:
<p>
<ol>
<li>Transform essay 1 into a list of *unique* words.
<li>Use Naive Bayes with that list of words and bag_of_essay1 to come up with a prediction.
<li>Transform essay 2 into a list of *unique* words.
<li>Use Naive Bayes with that list of words and bag_of_essay2 to come up with a prediction.
<li>Transform title into a list of *unique* words.
<li>Use Naive Bayes with that list of words and bag_of_title to come up with a prediction.
<li>Store the 3 predictions as a tuple and add to all_predictions.
</ol>
This took me roughly 6 minutes. In the end I had a list of triples. Each triple represents the 3 votes from our experts.

</div>

In [29]:
start = time.time()

all_predictions = []
count = 0
needed = ["project_title", "project_essay_1", "project_essay_2", "project_is_approved"]
for i in range(len(donate_table)):
    vec = donate_table.loc[i, needed]
    title = sentence_wrangler(vec["project_title"], swords, legals)[0]
    essay_1 = sentence_wrangler(vec["project_essay_1"], swords, legals)[0]
    essay_2 = sentence_wrangler(vec["project_essay_2"], swords, legals)[0]
    
    #get unique words
    u_t = []
    u_e1 = []
    u_e2 = []
    for word in title:
        if word not in essay_1 and word not in essay_2 and word not in u_t:
            u_t.append(word)
    for word in essay_1:
        if word not in essay_2 and word not in title and word not in u_e1:
            u_e1.append(word)
    for word in essay_2:
        if word not in essay_1 and word not in title and word not in u_e2:
            u_e2.append(word)
    print u_t 
    #get predictions
    t_predict = naive_bayes(u_t, bag_of_title, useful_counts)
    if t_predict[0] >= t_predict[1]:
        t_result = 0
    else:
        t_result = 1
    e1_predict = naive_bayes(u_e1, bag_of_essay1, useful_counts)
    if e1_predict[0] >= e1_predict[1]:
        e1_result = 0
    else:
        e1_result = 1
    e2_predict = naive_bayes(u_e2, bag_of_essay2, useful_counts)
    if e2_predict[0] >= e2_predict[1]:
        e2_result = 0
    else:
        e2_result = 1
    results = (e1_result, e2_result, t_result)
    if i == 12: break
    all_predictions.append(results)
    if i%4000 == 0: 
        count += 1
        print('%d/46' % count)
            
end = time.time()
print(end - start)  # roughly 6 minutes

[u'super', u'centers']
1/46
[u'keep', u'calm']
[u'lets', u'3doodle']
[u'gain']
[u'need', u'clean', u'culinary', u'arts']
[u'reach']
[u'kindergartners']
[u'fabulous', u'firsties', u'wiggling']
[u'fidgety', u'kids', u'focus']
[]
[u'writing']
[]
[u'literacy']
0.047455072403


In [30]:
all_predictions[:10]

[(1, 1, 1),
 (1, 0, 1),
 (1, 1, 1),
 (1, 1, 1),
 (1, 1, 1),
 (1, 1, 1),
 (1, 1, 1),
 (1, 1, 1),
 (1, 1, 1),
 (1, 1, 1)]

<h2>
Take the vote to get majority rules
</h2>
<div class=h1_cell>
<p>

Generate a new list combined_predictions that represents the majority vote in each triple

</div>

In [70]:
combined_predictions = []
for triple in all_predictions:
    ones = 0
    zeros = 0
    for prediction in triple:
        if prediction == 1:
            ones += 1
        else:
            zeros += 1
    if ones > zeros:
        combined_predictions.append(1)
    else:
        combined_predictions.append(0)


In [71]:
combined_predictions[:10]  # should match up with majority in all_prediction triples

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [72]:
actuals = []
for i in range(len(donate_table)):
    actuals.append(donate_table.loc[i, "project_is_approved"])

zipped = zip(combined_predictions, actuals)

In [73]:
1.0*(zipped.count((0,0)) + zipped.count((1,1)))/len(donate_table)  # trying to beat 0.8476823374340949

0.862197934973638

In [74]:
confusion_dictionary = {(1,1):0, (1,0):0, (0,1):0, (0,0):0}
for pair in zipped:
    confusion_dictionary[pair] += 1
confusion_dictionary

{(0, 0): 2887, (0, 1): 244, (1, 0): 24847, (1, 1): 154102}

<h2>
Hmmm - nothing to write home about
</h2>
<div class=h1_cell>
<p>
The actual percentate of accepts is 0.8476823374340949. So if we just always predict 1, we will get that. We are beating that by 1 percentage point roughly. As you can see we are doing poorly with false-positives.

</div>

<h2>
Let's look at our experts indivually
</h2>
<div class=h1_cell>
<p>
Just take the votes of the NB for essay 1.

</div>

In [76]:
needed = ["project_title", "project_essay_1", "project_essay_2", "project_is_approved"]
just_essay1 = []
just_essay2 = []
just_title = []
count = 0
for i in range(len(donate_table)):
    vec = donate_table.loc[i, needed]
    e1 = naive_bayes(vec['project_essay_1'], bag_of_essay1, useful_counts)
    if e1[0] >= e1[1]:
        just_essay1.append(0)
    else:
        just_essay1.append(1)
    e2 = naive_bayes(vec['project_essay_2'], bag_of_essay2, useful_counts)
    if e2[0] >= e2[1]:
        just_essay2.append(0)
    else:
        just_essay2.append(1)
    title = naive_bayes(vec['project_title'], bag_of_title, useful_counts)
    if title[0] >= title[1]:
        just_title.append(0)
    else:
        just_title.append(1)

    if i%4000 == 0: 
        count += 1
        print('%d/46' % count)

1/46
2/46
3/46
4/46
5/46
6/46
7/46
8/46
9/46
10/46
11/46
12/46
13/46
14/46
15/46
16/46
17/46
18/46
19/46
20/46
21/46
22/46
23/46
24/46
25/46
26/46
27/46
28/46
29/46
30/46
31/46
32/46
33/46
34/46
35/46
36/46
37/46
38/46
39/46
40/46
41/46
42/46
43/46
44/46
45/46
46/46


In [77]:
zipped = zip(just_essay1, actuals)
1.0*(zipped.count((0,0)) + zipped.count((1,1)))/len(donate_table)

0.8501757469244289

<h2>
A bit worse
</h2>
<div class=h1_cell>
<p>
Just take the votes of the NB for essay 2.

</div>

In [78]:
zipped = zip(just_essay2, actuals)
1.0*(zipped.count((0,0)) + zipped.count((1,1)))/len(donate_table)

0.870161467486819

<h2>
A bit better than the ensemble.
</h2>
<div class=h1_cell>
<p>
Just take the votes of the NB for title.

</div>

In [79]:
zipped = zip(just_title, actuals)
1.0*(zipped.count((0,0)) + zipped.count((1,1)))/len(donate_table)

0.8570188927943762

<h2>
Ok, that is not great
</h2>
<div class=h1_cell>
<p>
Have to think about whether want to include the vote for title.

</div>

<h2>
Closing notes
</h2>
<div class=h1_cell>
<p>
The ensemble we have put together is not very strong. Does that mean an ensemble approach won't work for this problem? I'd say the opposite. I bet it will. But not looking only at the text fields. My winter quarter data science class looked at non-text columns for use in prediction. In fact, there are a number of non-text columns that look promising, i.e., teacher_prefix,	school_state,	project_submitted_datetime,	project_grade_category,	project_subject_categories,	project_subject_subcategories, teacher_number_of_previously_posted_projects. I could use decision trees, a random forest or KNN to focus on some of these columns. I think I could have a very powerful ensemble if I expanded what I have with the 3 text columns: add methods that looked at more columns. Use all the info that is available to me, not just the text information.
<p>
The take-home message is that so far, no one machine learning method has proven to beat all others in all conditions. The ensemble approach attempts to let experts take on pieces of the problem and then combine their results. Naive Bayes is good at text analysis. Now add methods that are good with other types of columns.

</div>